In [1]:
import os
import numpy as np
import pandas as pd
import pickle
import sklearn.linear_model
from sklearn.model_selection import GridSearchCV
import sklearn.pipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
import sklearn.metrics


# Import our custom functions
from load_data import load_data 

In [2]:
x_tr, y_tr_df = load_data('x_train.csv', 'y_train.csv')
x_va, y_va_df = load_data('x_valid.csv', 'y_valid.csv')
x_te = load_data('x_test.csv', 'y_valid.csv')[0]


for label, arr in [('train', x_tr), ('valid', x_va)]:
    print("Contents of %s_x.csv: arr of shape %s" % (
        label, str(arr.shape)))

Contents of train_x.csv: arr of shape (2102, 784)
Contents of valid_x.csv: arr of shape (600, 784)


# Model with Duplicate Data

In [3]:
#getting index for each class in training set:
top_int = y_tr_df.index[y_tr_df['class_name']=='top']    #index 1
trous_int = y_tr_df.index[y_tr_df['class_name']=='trouser']    #index 11
dress_int =(y_tr_df.index[y_tr_df['class_name']=='dress']).tolist()
pull_int =(y_tr_df.index[y_tr_df['class_name']=='pullover']).tolist()
sneaker_int =(y_tr_df.index[y_tr_df['class_name']=='sneaker']).tolist()
sandal_int =(y_tr_df.index[y_tr_df['class_name']=='sandal']).tolist()


#sneakers + sandals + tops + trous + dress + pullovers in new dataset.
train_x_2 = np.vstack((x_tr[sandal_int, :], x_tr[sneaker_int, :]))
tops800= np.tile(x_tr[top_int, :], (800, 1)) #make 800 copies of tops
train_x_2 = np.vstack((train_x_2, tops800)) # add 800 tops to sandals + sneakers
assert(train_x_2.shape == (2400, 784))  #checking dimensions

trous800 = np.tile(x_tr[trous_int, :], (800, 1))  #800 copies of trousers
train_x_2 = np.vstack((train_x_2, trous800)) # add 800 trousers
assert(train_x_2.shape == (3200, 784))  #checking dimensions

dress800 = np.tile(x_tr[dress_int, :], (2, 1))   #doubled dresses 400 to 800
assert(dress800.shape == (800, 784))
train_x_2 = np.vstack((train_x_2, dress800)) # add 800 dresses
assert(train_x_2.shape == (4000, 784))  #checking dimensions

pull800 = np.tile(x_tr[pull_int, :], (8, 1))
assert(pull800.shape == (800, 784))
train_x_2 = np.vstack((train_x_2, pull800)) # add 800 dresses
assert(train_x_2.shape == (4800, 784))

#making train_y_2
train_y_2 = np.tile(7, 800) 
train_y_2 = np.hstack((train_y_2, np.tile(5, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(0, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(1, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(3, 800)))
train_y_2 = np.hstack((train_y_2, np.tile(2, 800)))

assert(train_y_2.shape == (4800, ))


In [4]:
# Combine training and validation datasets
x_dup_all = np.vstack((train_x_2, x_va))
y_dup_all = np.hstack((train_y_2, y_va_df['class_uid']))

print("Training X shape: %s\nValidation X shape: %s\nCombined X shape: %s\n" % (train_x_2.shape, x_va.shape, x_dup_all.shape))
print("Training Y shape: %s\nValidation Y shape: %s\nCombined Y shape: %s\n" % (train_y_2.shape, y_va_df['class_uid'].shape, y_dup_all.shape))

valid_dup_indicators = np.hstack([
    -1 * np.ones(train_y_2.shape[0]), #-1 = exclude this example in test split
    0 * np.ones(y_va_df.shape[0]), #0 = include in the first test split
    ])

# Define custom splitter to use only the validation dataset for hyperparameter selection
print("Splitter dimensions: %i" % (valid_dup_indicators.shape[0]))
my_dup_splitter = sklearn.model_selection.PredefinedSplit(valid_dup_indicators)

Training X shape: (4800, 784)
Validation X shape: (600, 784)
Combined X shape: (5400, 784)

Training Y shape: (4800,)
Validation Y shape: (600,)
Combined Y shape: (5400,)

Splitter dimensions: 5400


## Random Search

In [8]:
flag = 'stop'
filename = '1D_initial_RanSearch.sav'

if os.path.isfile("./" + filename) AND flag != 'run'::
    ran_dup_fashion_pipes = pickle.load(open(filename, 'rb'))
else:
    #batch size only for non-lbfgs. 
    #Learning Rate = const, adaptive, etc... ONLY for sgd
    #early_stopping only for sgd/adam.
    ran_params = dict(alpha = np.logspace(-5, 5, 100), learning_rate_init=np.logspace(-5, 5, 100), max_iter = [50, 100, 200, 300], hidden_layer_sizes=[(10,), (20,),(50,),(100,),(200,),(500,)])

    ran_dup_fashion_pipes = sklearn.pipeline.Pipeline([
        ('rand_search', RandomizedSearchCV(sklearn.neural_network.MLPClassifier (activation='identity', solver='lbfgs', shuffle=True, random_state=0), ran_params, scoring='balanced_accuracy', error_score='raise', return_train_score=True, n_iter=100, cv= my_dup_splitter, n_jobs = -1, refit= False, random_state=0))
    ])

    ran_dup_fashion_pipes.fit(x_dup_all, y_dup_all)
    
    pickle.dump(ran_dup_fashion_pipes, open(filename, 'wb'))

In [9]:
#loading the pickled modle!
load_initial = pickle.load(open('1D_initial_RanSearch.sav', 'rb'))
result = load_initial['rand_search'].cv_results_['rank_test_score']

In [11]:
best_index= np.argsort(result)
for i in best_index:
    score =load_initial['rand_search'].cv_results_['mean_test_score'][i]
    params = load_initial['rand_search'].cv_results_['params'][i]
    print("mean_test_score: %f" % (score), "params: %s" %params)
# the BEST score 0.565 mean test

mean_test_score: 0.565000 params: {'max_iter': 50, 'learning_rate_init': 0.03430469286314919, 'hidden_layer_sizes': (10,), 'alpha': 2420.1282647943835}
mean_test_score: 0.565000 params: {'max_iter': 300, 'learning_rate_init': 15556.761439304722, 'hidden_layer_sizes': (10,), 'alpha': 0.0006579332246575682}
mean_test_score: 0.565000 params: {'max_iter': 50, 'learning_rate_init': 7742.636826811277, 'hidden_layer_sizes': (10,), 'alpha': 0.8902150854450392}
mean_test_score: 0.565000 params: {'max_iter': 300, 'learning_rate_init': 0.2782559402207126, 'hidden_layer_sizes': (10,), 'alpha': 1.4174741629268048}
mean_test_score: 0.565000 params: {'max_iter': 300, 'learning_rate_init': 3853.5285937105355, 'hidden_layer_sizes': (10,), 'alpha': 7742.636826811277}
mean_test_score: 0.385000 params: {'max_iter': 300, 'learning_rate_init': 3053.8555088334124, 'hidden_layer_sizes': (500,), 'alpha': 0.08697490026177834}
mean_test_score: 0.383333 params: {'max_iter': 100, 'learning_rate_init': 236.44894126

In [18]:
flag = 'stop'
filename = '1D_2nd_RanSearch.sav'

if os.path.isfile("./" + filename) AND flag != 'run'::
    ran_dup_fashion_pipes2 = pickle.load(open(filename, 'rb'))
else:
    ran_params2 = dict(alpha = np.logspace(-5, 3, 100), learning_rate_init=np.logspace(-5, 5, 100), max_iter = [5, 10, 25, 26, 40, 290, 300, 310, 311, 312])

    ran_dup_fashion_pipes2 = sklearn.pipeline.Pipeline([
        ('rand_search', RandomizedSearchCV(sklearn.neural_network.MLPClassifier (activation='identity', solver='lbfgs', shuffle=True, random_state=0, hidden_layer_sizes= (10,)), ran_params2, scoring='balanced_accuracy', error_score='raise', return_train_score=True, n_iter=500, cv= my_dup_splitter, n_jobs = -1, refit= False, random_state=0))
    ])

    ran_dup_fashion_pipes2.fit(x_dup_all, y_dup_all)
    
    pickle.dump(ran_dup_fashion_pipes2, open(filename, 'wb'))

In [20]:
best_index= np.argsort(ran_dup_fashion_pipes2['rand_search'].cv_results_['rank_test_score'])
for i in best_index:
    score =ran_dup_fashion_pipes2['rand_search'].cv_results_['mean_test_score'][i]
    params = ran_dup_fashion_pipes2['rand_search'].cv_results_['params'][i]
    print("mean_test_score: %f" % (score), "params: %s" %params)
# the BEST score 0.565 mean test
#mean_test_score: 0.621667 params: {'max_iter': 40, 'learning_rate_init': 2.0092330025650458e-05, 'hidden_layer_sizes': (10,), 'alpha': 62802.914418342465}

mean_test_score: 0.565000 params: {'max_iter': 311, 'learning_rate_init': 0.0020565123083486517, 'alpha': 3.1257158496882416}
mean_test_score: 0.565000 params: {'max_iter': 5, 'learning_rate_init': 0.0008111308307896872, 'alpha': 0.23101297000831628}
mean_test_score: 0.565000 params: {'max_iter': 300, 'learning_rate_init': 0.00014174741629268049, 'alpha': 1000.0}
mean_test_score: 0.565000 params: {'max_iter': 26, 'learning_rate_init': 0.4328761281083057, 'alpha': 0.029836472402833405}
mean_test_score: 0.565000 params: {'max_iter': 40, 'learning_rate_init': 0.34304692863149194, 'alpha': 29.15053062825182}
mean_test_score: 0.565000 params: {'max_iter': 26, 'learning_rate_init': 0.005214008287999685, 'alpha': 0.7054802310718645}
mean_test_score: 0.565000 params: {'max_iter': 5, 'learning_rate_init': 0.00031992671377973844, 'alpha': 6.428073117284319e-05}
mean_test_score: 0.565000 params: {'max_iter': 311, 'learning_rate_init': 0.0001, 'alpha': 0.011768119524349991}
mean_test_score: 0.5650

## Grid Search

In [21]:
distEvalHype = dict(learning_rate_init = np.logspace(-3, 2, 50), alpha = np.logspace(-5, 3, 50), random_state = [0, 1, 2], max_iter = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

grid_dup_fashion_pipes = sklearn.pipeline.Pipeline([
('grid_search', GridSearchCV(sklearn.neural_network.MLPClassifier
                             (activation='identity', solver='lbfgs', learning_rate= 'constant', shuffle=True, early_stopping = True, random_state=0, hidden_layer_sizes = (10, )), distEvalHype,
                              scoring='balanced_accuracy', error_score='raise',
                              return_train_score=True, cv= my_dup_splitter, n_jobs = -1, refit= False))
])
grid_dup_fashion_pipes.fit(x_dup_all, y_dup_all)


# filename = '1D_second_RanSearch.sav'
# pickle.dump(dup_fashion_pipes, open(filename, 'wb'))

Pipeline(steps=[('grid_search',
                 GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
                              error_score='raise',
                              estimator=MLPClassifier(activation='identity',
                                                      early_stopping=True,
                                                      hidden_layer_sizes=(10,),
                                                      random_state=0,
                                                      solver='lbfgs'),
                              n_jobs=-1,
                              param_grid={'alpha': array([1.00000000e-05, 1.45634848e-05, 2.12095089e-05, 3.08884360e-05,
       4.49843267e-05...
       1.84206997e+00, 2.32995181e+00, 2.94705170e+00, 3.72759372e+00,
       4.71486636e+00, 5.96362332e+00, 7.54312006e+00, 9.54095476e+00,
       1.20679264e+01, 1.52641797e+01, 1.93069773e+01, 2.44205309e+01,
       3.08884360e+01, 3.90693994e+01, 4.94171336e+01, 6.25055193e+01,
       7.90604321e+01, 1.00000000e+02]),
                                          'max_iter': [1, 2, 3, 4, 5, 6, 7, 8,
                                                       9, 10],
                                          'random_state': [0, 1, 2]},
                              refit=False, return_train_score=True,
                              scoring='balanced_accuracy'))])

In [22]:
pickle.dump(grid_dup_fashion_pipes, open('1D_second_RanSearch.sav', 'wb'))

In [25]:
#loading the pickled modle!
load_low_grid = pickle.load(open('1D_second_RanSearch.sav', 'rb'))
result = load_low_grid['grid_search'].cv_results_

In [29]:
best_grid_index= np.argsort(load_low_grid['grid_search'].cv_results_['rank_test_score'])
for i in best_grid_index:
    low_grid_score =load_low_grid['grid_search'].cv_results_['mean_test_score'][i]
    low_grid_params = load_low_grid['grid_search'].cv_results_['params'][i]
    print("mean_test_score: %f" % (low_grid_score), "params: %s" %low_grid_params)
# the BEST score 0.565 mean test
#mean_test_score: 0.621667 params: {'max_iter': 40, 'learning_rate_init': 2.0092330025650458e-05, 'hidden_layer_sizes': (10,), 'alpha': 62802.914418342465}

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices